### Задание

Используя XGBOOST предсказать размер чаевых в датасете TIPS

In [1]:
import seaborn as sns
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()

/home/liza/env/env/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
FILEPATH = "./tips.txt"

df = pd.read_csv(FILEPATH).dropna()
dfTips = df
dfTips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [3]:
days = {'Mon':0, 'Tue':1, 'Wed':2, 'Thur':3, 'Fri':4, 'Sat':5, 'Sun':6}
dfTips['day'] = dfTips['day'].map(days)

dfTips['sex'] = ord_enc.fit_transform(dfTips[["sex"]]) # m=1 f=0
dfTips['smoker'] = ord_enc.fit_transform(dfTips[["smoker"]]) #t=1 f=0
dfTips['time'] = ord_enc.fit_transform(dfTips[["time"]]) #d=0 l=1

dfTips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0.0,0.0,6,0.0,2
1,10.34,1.66,1.0,0.0,6,0.0,3
2,21.01,3.50,1.0,0.0,6,0.0,3
3,23.68,3.31,1.0,0.0,6,0.0,2
4,24.59,3.61,0.0,0.0,6,0.0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,1.0,0.0,5,0.0,3
240,27.18,2.00,0.0,1.0,5,0.0,2
241,22.67,2.00,1.0,1.0,5,0.0,2
242,17.82,1.75,1.0,0.0,5,0.0,2


In [4]:
dfY = dfTips.iloc[:,1:2]
dfX = dfTips.drop(['tip'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(dfX, dfY)

In [6]:
regress = xgb.XGBRegressor( 
    base_score = 0.1,
    n_estimators=100,
    max_depth=5 
)
regress.fit(X_train, y_train)

/home/liza/env/env/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.1, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [8]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

pd.DataFrame(regress.feature_importances_.reshape(1, -1), columns=dfX.columns)

y_pred = regress.predict(X_test)

print('score: '+ str(regress.score(X_test , y_test)))
print('squered error: '+ str(mean_squared_error(y_test, y_pred)))

score: 0.15914566605562064
squered error: 1.1438609951577474


In [11]:
y_pred

array([1.3853676, 5.5369935, 3.1764631, 2.8772388, 1.8408371, 2.4422972,
       2.6165104, 1.8495766, 2.5553634, 2.4841368, 1.5109009, 3.16786  ,
       1.55136  , 4.6950436, 4.0201516, 3.0944874, 4.5558057, 3.5923185,
       1.8840444, 2.7849379, 3.0339794, 4.542726 , 2.0079427, 2.5764916,
       2.1348093, 3.1738174, 2.5425174, 2.557273 , 2.6980288, 4.543484 ,
       3.0255814, 4.520283 , 6.2347426, 2.8677301, 1.7022272, 3.2362506,
       2.7625494, 1.5401578, 2.0099294, 2.3830054, 3.6735682, 2.6976101,
       2.9034033, 1.7704431, 3.531671 , 3.4380698, 3.0263855, 3.1326735,
       1.7705865, 1.8375367, 2.8873477, 1.503858 , 5.8747625, 3.1092622,
       2.4699495, 1.5332137, 2.9200962, 2.5831656, 2.8061805, 1.4497046,
       1.8375367], dtype=float32)